In [1]:
# import necessary packages
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import geopandas as gpd
from descartes import PolygonPatch
from tqdm import tqdm

In [2]:
df1 = pd.read_csv("data/full_run_1.zip",skiprows=6)
df2 = pd.read_csv("data/full_run_2.zip",skiprows=6)
df3 = pd.read_csv("data/full_run_3.zip",skiprows=6)
df4 = pd.read_csv("data/full_run_4.zip",skiprows=6)

In [5]:
for dfs in [df1,df2,df3,df4]:
    print(dfs.shape)

(10496, 42)
(10496, 42)
(10496, 42)
(11808, 42)


In [26]:
#remove all rows with no data, these are usually the rows at the start of a run.
#make sure to change the numbers here, so df1-df4 are done. A for loop doesn't work here!
l = []
for r in df4.index:
    if len(df4.iloc[r,22:].unique())<2:
        l.append(r)

df4 = df4.drop(index=l)

In [6]:
df = pd.concat([df1,df2,df3,df4],0)
df.shape

(43296, 42)

In [32]:
#I found this code to make dict from long list.
def listToDict(lst):
    op = {lst[i]: lst[i + 1] for i in range(0, len(lst), 2)}
    return op

In [34]:
#This will take two eons of time, please don't run this code unless you really need to.
steps = []
n_steps = 6969696969 
for index, row in tqdm(df.iloc[:n_steps,:].iterrows(), total=n_steps):
    l = []
    for column in df.columns[:22]:
        x = row[column]
        new_df = pd.DataFrame(index=np.arange(0,114), columns=[column])
        new_df[column] = x
        l.append(new_df)
    for column in df.columns[22:]:
        x = row[column].replace("[","").replace("]","").split(" ")
        d = listToDict(x)
        new_df = pd.DataFrame(index=np.arange(0, len(d)), columns=['Neighborhood number', column])
        new_df['Neighborhood number'] = [*d]
        new_df[column] = list(d.values())
        l.append(new_df)
        # empty_df.loc[index, column] = x
    steps.append(pd.concat(l, axis=1))

  0%|                                                                | 42240/6969696969 [37:05<101982:52:48, 18.98it/s]


In [35]:
full_df = pd.concat(steps).loc[:,~pd.concat(steps).columns.duplicated()].convert_dtypes()

In [36]:
full_df.dtypes

[run number]                          Int64
size-of-std                         float64
transform-houses?                   boolean
avg_migrant_income                    Int64
amount-of-health-improved             Int64
aggregate-cluster-size                Int64
improve-health?                     boolean
amount-of-mixed-nhs                   Int64
citizen-color                        string
build-percent                       float64
amount-of-safety-improved             Int64
recolor-agents?                     boolean
increase-social-housing?            boolean
build-more-houses?                  boolean
random-attributes?                  boolean
increase-percent                      Int64
housing-market-inflation?           boolean
transformed-percentage                Int64
migrant-multiplier                  float64
mixed-use-zoning?                   boolean
improve-safety?                     boolean
[step]                                Int64
Neighborhood number             

In [40]:
full_df = full_df.astype({'Neighborhood number' : int,
       'KPI-available_buy_houses' : int, 'KPI-available_part_rent_houses' : float,
       'KPI-available_social_rent_houses' : int, 'KPI-avg_income' : float, 'KPI-avg_price' : float,
       'KPI-avg_utility' : float, 'KPI-citizen-count' : int, 'KPI-p-sc-lower' : float,
       'KPI-p-sc-working' : float, 'KPI-p-sc-middle' : float, 'KPI-p-sc-upper' : float, 'KPI-p-dutch' : float,
       'KPI-p-other-western' : float, 'KPI-p-antilles' : float, 'KPI-p-morocco' : float,
       'KPI-p-suriname' : float, 'KPI-p-turkey' : float, 'KPI-p-indonesian' : float,
       'KPI-p-eastern-eu' : float, 'KPI-p-other-nonwestern' : float})

In [43]:
#remove all the columns that have the same value in all rows.
for c in full_df.columns:
    if len(full_df[c].unique()) < 2:
        full_df = full_df.drop(columns=c)

In [47]:
full_df.to_csv("output/full_df.csv")